#1. Data

1.1 Load

In [1]:
from sklearn.preprocessing import Normalizer
import pandas as pd
import numpy as np

df = pd.read_csv('wheat.csv')
df.info()
# df = df.drop(np.random.choice(df.index, remove_n, replace=False))
# # df = df.reset_index(drop=True)

y = df.category.astype(int)


df = df.drop(columns='category')
print(df.head())

classes = set(y)
classes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   area                   210 non-null    float64
 1   perimeter              210 non-null    float64
 2   compactness            210 non-null    float64
 3   length                 210 non-null    float64
 4   width                  210 non-null    float64
 5   asymmetry coefficient  210 non-null    float64
 6   groove length          210 non-null    float64
 7   category               210 non-null    float64
dtypes: float64(8)
memory usage: 13.2 KB
    area  perimeter  compactness  length  width  asymmetry coefficient  \
0  15.26      14.84       0.8710   5.763  3.312                  2.221   
1  14.88      14.57       0.8811   5.554  3.333                  1.018   
2  14.29      14.09       0.9050   5.291  3.337                  2.699   
3  13.84      13.94       0.8955   5.32

{1, 2, 3}

1.2 Bootstrap

In [2]:
df['y_true'] = y
bts = []
for i in range(3):
  df_bt = df.sample(frac=0.5)
  y_bt = df_bt['y_true']
  X_bt = df_bt.drop(columns=['y_true'])
  bts.append([X_bt, y_bt])

1.3 Classification

In [3]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

models = [GaussianNB(), MLPClassifier(), SVC()]
preds= []
for i, model in enumerate(models):
  X_bt = bts[i][0]
  y_bt = bts[i][1]
  model.fit(X_bt, y_bt)
  y_pred = model.predict(X_bt)
  pred_df = pd.DataFrame(y_pred, index = X_bt.index)
  preds.append(pred_df)
  print(accuracy_score(y_bt, y_pred))

0.9333333333333333
0.9238095238095239
0.8761904761904762


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


1.4 Aggregation of predictions

In [4]:
preds_df = pd.concat(preds, axis=1)
preds_df.head()

,0,0,0
0,1.0,1.0,NaN
1,1.0,NaN,NaN
2,1.0,1.0,1.0
4,1.0,NaN,NaN
5,1.0,1.0,1.0


In [5]:
preds_df.to_csv('preds.csv')

#2. Ontology

**Epistemic and doxastic logic**

2.2 Creating ontology

In [6]:
!pip install owlready2
from owlready2 import *

onto = get_ontology('onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2.3 Creating classes and properties

In [7]:
import types

with onto:
  tsr = types.new_class('tsr', (ObjectProperty, TransitiveProperty, SymmetricProperty, ReflexiveProperty))
  ts = types.new_class('ts', (ObjectProperty, TransitiveProperty, SymmetricProperty))

  for cl in classes:
    onto_cl = types.new_class(f'Class{cl}', (Thing,))
    onto_cl_know = types.new_class(f'Know_Class{cl}', (Thing,))
    onto_cl_know.equivalent_to.append(tsr.only(onto_cl))
    onto_cl_believe = types.new_class(f'Believe_Class{cl}', (Thing,))
    onto_cl_believe.equivalent_to.append(ts.only(onto_cl))

onto.save('onto.owl', format='ntriples')

2.4 Creating instances

In [8]:
with onto:
  for index, row in preds_df.iterrows():
    inst_old = None
    for j, pred in enumerate(preds_df.columns):
      cl = row.iloc[j]
      if not pd.isna(cl):
        cl = int(cl)
        onto_cl = onto[f'Class{cl}']
        inst_new = onto_cl(f'inst{index}_{j}')
        if inst_old != None:
          inst_old.tsr.append(inst_new)
          inst_old.ts.append(inst_new)
        inst_old = inst_new

onto.save('onto_insts.owl', format='ntriples')

2.5 Inferring relations

In [9]:
with onto:
  sync_reasoner_pellet(infer_property_values=True, debug=2)

onto.save('onto_rels.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/l

* Owlready * Adding relation onto.inst208_0 tsr onto.inst208_0
* Owlready * Adding relation onto.inst208_0 ts onto.inst208_0
* Owlready * Adding relation onto.inst129_1 tsr onto.inst129_1
* Owlready * Adding relation onto.inst129_1 ts onto.inst129_1
* Owlready * Adding relation onto.inst26_1 tsr onto.inst26_1
* Owlready * Adding relation onto.inst26_1 ts onto.inst26_1
* Owlready * Adding relation onto.inst106_1 tsr onto.inst106_1
* Owlready * Adding relation onto.inst106_1 ts onto.inst106_1
* Owlready * Adding relation onto.inst100_1 tsr onto.inst100_1
* Owlready * Adding relation onto.inst100_1 ts onto.inst100_1
* Owlready * Adding relation onto.inst176_1 tsr onto.inst176_1
* Owlready * Adding relation onto.inst176_1 ts onto.inst176_1
* Owlready * Adding relation onto.inst159_1 tsr onto.inst159_1
* Owlready * Adding relation onto.inst159_1 ts onto.inst159_1
* Owlready * Adding relation onto.inst19_0 tsr onto.inst19_0
* Owlready * Adding relation onto.inst98_0 tsr onto.inst98_0
* Owlre

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


2.6 OWA/CWA

In [10]:
with onto:
  for inst in onto.individuals():
    close_world(inst)

2.7 Inferring classification

In [11]:
with onto:
  sync_reasoner_pellet(debug=2)

onto.save('onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/l